In [1]:
import pandas as pd
import numpy as np
import ahocorasick
from tqdm import tqdm
import glob
from Bio import SeqIO
import csv
import time

In [2]:
cd /Users/davidchen/Documents/GitHub/Sandor_David/Miniscreen05

/Users/davidchen/Documents/GitHub/Sandor_David/Miniscreen05


In [3]:
primers = pd.read_csv("Fw_staggered_primers.tsv", sep='\t', header=None)
rna = pd.read_csv("gRNAmotifs5.tsv", sep='\t', header=None)
directory = "/Users/davidchen/Documents/GitHub/Sandor_David/Miniscreen05/Sequence"

In [9]:
primers

,0,1
0,F1,TTGTGGAAAGGACGAAACACCG
1,F2,CTTGTGGAAAGGACGAAACACCG
2,F3,GCTTGTGGAAAGGACGAAACACCG
3,F4,AGCTTGTGGAAAGGACGAAACACCG
4,F5,CAACTTGTGGAAAGGACGAAACACCG
5,F6,TGCACCTTGTGGAAAGGACGAAACACCG
6,F7,ACGCAACTTGTGGAAAGGACGAAACACCG
7,F8,GAAGACCCTTGTGGAAAGGACGAAACACCG


In [10]:
rna

,0,1
0,Seq1,ATCTTGTGGAAAGGACGAAACACC
1,Vec,GTGGAAAGGACGAAACACCG
2,SOX9_g1,ACTTCCGCGACGTGGACAT
3,SOX9_g2,TCACCGACTTCCTCCGCCG
4,SOX9_g3,CCATGTCCGAGGACTCCGC
...,...,...
73,mKate2_g2,GTCACCACATACGAAGACGG
74,mKate2_g3,GCACATGAAGCTGTACATGG
75,mKate2_g4,GTGATGCAGAAGAAAACACT
76,mKate2_g5,ACCTTCATCAACCACACCCA


In [76]:
sequence_files = sorted([f for f in glob.glob(directory + "/*.fastq")])

In [130]:
def demultiplex(primers, directory, line):
    #Ahocorasick automaton
    A = ahocorasick.Automaton()
    for idx, key in enumerate(primers.iloc[:,1]):
        A.add_word(key, (idx, key))
    A.make_automaton()

    primer_dict = dict(zip(primers.iloc[:, 1], primers.iloc[:, 0]))
    
    print(str(line.split("/")[-1].split(".")[0]))
    
    #Iterate through each read in a given fastq file
    with open(line, "r") as handle:
        
        for sequence in tqdm(list(SeqIO.parse(handle, "fastq"))):
        #for sequence in tqdm(SeqIO.parse(handle, "fastq")):
            
            seq_string = str(sequence.seq)
            seq_id = ">" + str(sequence.id)
            
            match_list = []
            for end_index, (insert_order, original_value) in A.iter(seq_string):
                match_list.append(original_value)

            
            if len(match_list) > 0:
                
                with open(directory + "/Split_Fraction/" + line.split("/")[-1].split(".")[0] + "_" + primer_dict[max(match_list)] + ".fasta", "a+") as file_object:
                    file_object.write(seq_id)
                    file_object.write("\n")
                    file_object.write(seq_string)
                    file_object.write("\n")

            else:
                
                with open(directory + "/Split_Fraction/" + line.split("/")[-1].split(".")[0] + "_unknown.fasta", "a+") as file_object:
                    file_object.write(seq_id)
                    file_object.write("\n")
                    file_object.write(seq_string)
                    file_object.write("\n")
                    

In [128]:
def demultiplex_old(primers, directory, m_mismatch, line):

    start = time.time()
    with open(directory + "/Split_Fraction/" + line.split("/")[-1].split(".")[0] + "_log.csv", "a+", newline='') as fp:
        wr = csv.writer(fp, dialect='excel')
        wr.writerow(primers.iloc[:, 0])
    
    #Iterate through each read in a given fastq file
    with open(line, "r") as handle:
        
        print(str(line.split("/")[-1].split(".")[0]))
        for sequence in tqdm(list(SeqIO.parse(handle, "fastq"))):
        #for sequence in tqdm(SeqIO.parse(handle, "fastq")):
            
            seq_string = str(sequence.seq)
            seq_id = ">" + str(sequence.id)
            
            #Iterate through each of the primers and check if there is a match 
            n_matches = [0,0,0,0,0,0,0,0]
            for primer in range(len(primers)):
                
                if m_mismatch == 0:
                
                    if primers.iloc[:,1][primer] in seq_string:
                        n_matches[primer] = 1
                        
                        #Output each file into sub-files based on index matches
                        with open(directory + "/Split_Fraction/" + line.split("/")[-1].split(".")[0] + "_" + primers.iloc[:,0][primer] + ".fasta", "a+") as file_object:
                            file_object.write(seq_id)
                            file_object.write("\n")
                            file_object.write(seq_string)
                            file_object.write("\n")
                        
                        break 
                            
                    else:
                        pass
                        
                else: 
                    
                    if len(regex.findall("(" + primers.iloc[:,1][primer] + "){s<=" + str(m_mismatch) + "}", seq_string)) >= 1:
                        #Record number of matches in sequence for each index 
                        n_matches[primer] = len(regex.findall("(" + primers.iloc[:,1][primer] + "){s<=" + str(m_mismatch) + "}", seq_string))
                        
                        #Output each file into sub-files based on index matches
                        with open(directory + "/Split_Fraction/" + line.split("/")[-1].split(".")[0] + "_" + primers.iloc[:,0][primer] + ".fasta", "a+") as file_object:
                            file_object.write(seq_id)
                            file_object.write("\n")
                            file_object.write(seq_string)
                            file_object.write("\n")
                    
                    else:
                        pass
                    
            
            #Write sequence with no matches to known primers to standalone sub-file
            if sum(n_matches) < 1:
                with open(directory + "/Split_Fraction/" + line.split("/")[-1].split(".")[0] + "_unknown.fasta", "a+") as file_object:
                    file_object.write(seq_id)
                    file_object.write("\n")
                    file_object.write(seq_string)
                    file_object.write("\n")
            else:
                pass
            
            with open(directory + "/Split_Fraction/" + line.split("/")[-1].split(".")[0] + "_log.csv", "a+", newline='') as fp:
                wr = csv.writer(fp, dialect='excel')
                wr.writerow(n_matches)

    print(str(line.split("/")[-1].split(".")[0]))
    
    end = time.time()
    print(end - start)

In [133]:
primer_sequence_files = sorted([f for f in glob.glob(directory + "/Split_Fraction/*.fasta")])

In [135]:
primer_sequence_files[0]

'/Users/davidchen/Documents/GitHub/Sandor_David/Miniscreen05/Sequence/Split_Fraction/20210204_MNSC501_SS8614_S1_R1_001_F1.fasta'

In [136]:
read_count_df = pd.DataFrame(data = 0, index =[x.split("/")[-1].split(".")[0]for x in primer_sequence_files], columns = rna.iloc[:, 0])

In [140]:
rna_dict = dict(zip(rna.iloc[:, 1], rna.iloc[:, 0]))

In [151]:
def read_count(primer_sequence_files, rna, directory, fraction):

    read_count_df = pd.DataFrame(data = 0, index =[x.split("/")[-1].split(".")[0]for x in primer_sequence_files], columns = rna.iloc[:, 0])
    rna_dict = dict(zip(rna.iloc[:, 1], rna.iloc[:, 0]))
    
    A = ahocorasick.Automaton()
    for idx, key in enumerate(rna.iloc[:,1]):
        A.add_word(key, (idx, key))
    A.make_automaton()
    
    with open(fraction, "r") as handle:
        
        print(fraction.split("/")[-1].split(".")[0])
        
        for sequence in tqdm(list(SeqIO.parse(handle, "fasta"))[:1000]):

            seq_string = str(sequence.seq)
            
            match_list = []
            for end_index, (insert_order, original_value) in A.iter(seq_string):
                match_list.append(original_value)
                
                start_index = end_index - len(original_value) + 1
                assert seq_string[start_index:start_index + len(original_value)] == original_value
            
            if len(match_list) > 0:
                read_count_df.loc[fraction.split("/")[-1].split(".")[0], rna_dict[max(match_list)]] += 1
                
            else:
                pass
            
    read_count_df.to_csv(directory + "/Split_Fraction_Read_Count/" + fraction.split("/")[-1].split(".")[0] + "_read_counts.csv", index=True)
    

In [152]:
def read_count_old(primer_sequence_files, rna, directory, n_mismatch, fraction):

    read_count_df = pd.DataFrame(data = 0, index =[x.split("/")[-1].split(".")[0]for x in primer_sequence_files], columns = rna.iloc[:, 0])
 
    with open(fraction, "r") as handle:
        
        print(fraction.split("/")[-1].split(".")[0])
        
        for sequence in tqdm(list(SeqIO.parse(handle, "fasta"))[:1000]):

            seq_string = str(sequence.seq)
            for rna_seq in range(len(rna.iloc[:,1])):
                
                if n_mismatch == 0:
                    if rna.iloc[:,1][rna_seq] in seq_string:
                        read_count_df.loc[fraction.split("/")[-1].split(".")[0], rna.iloc[:,0][rna_seq]] += 1
                    else:
                        pass
                
                else: 
                    if len(regex.findall("(" + rna.iloc[:,1][rna_seq] + "){s<=" + str(n_mismatch) + "}", seq_string)) >= 1:
                        read_count_df.loc[fraction.split("/")[-1].split(".")[0], rna.iloc[:,0][rna_seq]] += 1
                    else:
                        pass
                    
    read_count_df.to_csv(directory + "/Split_Fraction_Read_Count/" + fraction.split("/")[-1].split(".")[0] + "_read_counts.csv", index=True)
    

In [83]:
A = ahocorasick.Automaton()
for idx, key in enumerate(primers.iloc[:,1]):
    A.add_word(key, (idx, key))
A.make_automaton()


In [84]:
haystack = str(next(SeqIO.parse(directory + "/20210204_MNSC501_SS8614_S1_R1_001.fastq", "fastq")).seq)
match_list = []
index = []
for end_index, (insert_order, original_value) in A.iter(haystack):
    match_list.append(original_value)
    index.append


    

In [92]:
primer_dict = dict(zip(primers.iloc[:, 1], primers.iloc[:, 0]))

In [94]:
primer_dict[max(match_list)]

'F1'

In [90]:
list(primers.iloc[:,1])

['TTGTGGAAAGGACGAAACACCG',
 'CTTGTGGAAAGGACGAAACACCG',
 'GCTTGTGGAAAGGACGAAACACCG',
 'AGCTTGTGGAAAGGACGAAACACCG',
 'CAACTTGTGGAAAGGACGAAACACCG',
 'TGCACCTTGTGGAAAGGACGAAACACCG',
 'ACGCAACTTGTGGAAAGGACGAAACACCG',
 'GAAGACCCTTGTGGAAAGGACGAAACACCG']

In [85]:
match_list

['TTGTGGAAAGGACGAAACACCG']

In [86]:
insert_order

0

In [87]:
original_value

'TTGTGGAAAGGACGAAACACCG'

In [66]:
end_index

40

In [70]:
haystack[:].index('GGCCGGGTGCCCAAGTCCC')

22

In [78]:
statDF.iloc[0,0] += 1

In [79]:
statDF

,Seq1,Vec,SOX9_g1,SOX9_g2,SOX9_g3,SOX9_g4,SOX9_g5,SOX9_g6,PROM1_g1,PROM1_g2,...,EGFP_g3,EGFP_g4,EGFP_g5,EGFP_g6,mKate2_g1,mKate2_g2,mKate2_g3,mKate2_g4,mKate2_g5,mKate2_g6
20210204_MNSC501_SS8614_S1_R1_001_F1,1,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
20210204_MNSC501_SS8614_S1_R1_001_F2,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
20210204_MNSC501_SS8614_S1_R1_001_F3,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
20210204_MNSC501_SS8614_S1_R1_001_F4,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
20210204_MNSC501_SS8614_S1_R1_001_F5,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
20210204_MNSC512_SS8614_S12_R1_001_F5,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
20210204_MNSC512_SS8614_S12_R1_001_F6,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
20210204_MNSC512_SS8614_S12_R1_001_F7,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
20210204_MNSC512_SS8614_S12_R1_001_F8,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
